# StayEstimate - Data Cleaning
Download, merge, and clean the data 



### Data Source: Health Data NY
Hospital Inpatient Discharges (SPARCS De-Identified):
<a href="https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/q6hk-esrj">2009 | </a>
<a href="https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/mtfm-rxf4">2010 | </a>
<a href="https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/pyhr-5eas">2011 | </a>
<a href="https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/u4ud-w55t">2012 | </a>
<a href="https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/npsr-cm47">2013</a>

## Initialize Paths

In [1]:
#Input Path
Input='Input'
#Output Path
Output='Output'
#Transitional Path
Trans='Transitional'

## Import Libraries 

In [2]:
#######      Libraries    #######
from os.path import join
import pandas as pd
from pandas import DataFrame, Series

# Data Cleaning


###  Load, Merge, and Clean

#### load 5 years

In [4]:
dfs=[]
for y in range(2009,2014):
    dfs.append(pd.read_csv(join(Input,'Hospital_Inpatient_Discharges__SPARCS_De-Identified___%d.csv'%y)
                                 ,low_memory=False))
    print "Year %d is Loaded!"%y

Year 2009 is Loaded!
Year 2010 is Loaded!
Year 2011 is Loaded!
Year 2012 is Loaded!
Year 2013 is Loaded!


#### merge the 5 years

In [5]:
#merge 5 years
df_5yr=pd.concat(dfs)

#### show all columns before cleaning

In [6]:
df_5yr.columns

Index([u'APR DRG Code', u'APR DRG Description', u'APR MDC Code',
       u'APR MDC Description', u'APR Medical Surgical Description',
       u'APR Risk of Mortality', u'APR Severity of Illness Code',
       u'APR Severity of Illness Description', u'Abortion Edit Indicator',
       u'Admit Day of Week', u'Age Group',
       u'Attending Provider License Number', u'Birth Weight',
       u'CCS Diagnosis Code', u'CCS Diagnosis Description',
       u'CCS Procedure Code', u'CCS Procedure Description',
       u'Discharge Day of Week', u'Discharge Year',
       u'Emergency Department Indicator', u'Ethnicity', u'Facility ID',
       u'Facility Id', u'Facility Name', u'Gender', u'Health Service Area',
       u'Hospital County', u'Length of Stay', u'Operating Certificate Number',
       u'Operating Provider License Number', u'Other Provider License Number',
       u'Patient Disposition', u'Race', u'Source of Payment 1',
       u'Source of Payment 2', u'Source of Payment 3', u'Total Charges',
      

#### Cleanup the Data

In [13]:
#Change "120+" string to 121 integer for simplicity
df_5yr['Stay_Length']=df_5yr['Length of Stay'].replace(['120 +'],[121]).astype('int')

#create a cleaned df with relevant columns
df_5yr_cleaned=df_5yr[['Gender','Age Group','Race','APR Severity of Illness Description',
                       'APR Risk of Mortality','APR DRG Description','APR Medical Surgical Description',
                       'Type of Admission','Admit Day of Week','Patient Disposition',
                       'Discharge Day of Week','Total Charges','Discharge Year','Stay_Length']]
#index as Diagnosis
df_5yr_cleaned.index=df_5yr['CCS Diagnosis Description']

#shorten column names
df_5yr_cleaned.rename(columns={'Age Group':'Age','APR Severity of Illness Description':'Severity',
                   'APR Medical Surgical Description':'MedicalSurgical','Discharge Year':'Year',
                   'Type of Admission':'Admission','APR Risk of Mortality':'Mortality',
                   'APR DRG Description':'DRG','Admit Day of Week':'AdmitDay','Discharge Day of Week':'DischDay',
                   'Patient Disposition':'Disposition','Total Charges':'Total_Charges'} ,inplace=True)

In [14]:
#drop missing data
df_5yr_cleaned.dropna(inplace=True)
df_5yr_cleaned=(df_5yr_cleaned[(df_5yr_cleaned['Gender']!='U')
   &(df_5yr_cleaned['Admission']!='Not Available')   
   &(df_5yr_cleaned['Race']!='Unknown')
   &(pd.notnull(df_5yr_cleaned.index))
  ])

print "Number of records in Original dataset:%d"%len(df_5yr)
print "Number of records in Cleaned dataset:%d"%len(df_5yr_cleaned)

Number of records in Original dataset:12820613
Number of records in Cleaned dataset:12694980


/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### show all columns after cleaning

In [15]:
df_5yr_cleaned.columns

Index([u'Gender', u'Age', u'Race', u'Severity', u'Mortality', u'DRG',
       u'MedicalSurgical', u'Admission', u'AdmitDay', u'Disposition',
       u'DischDay', u'Total_Charges', u'Year', u'Stay_Length'],
      dtype='object')

### Store Cleaned Data 

In [24]:
df_5yr_cleaned.to_csv(join(Trans,'df_5yr_cleaned.csv'))